# Mask R-CNN - Train modified model on Shapes Dataset

### the modified model does not include any mask related heads or losses 



In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import sys
import random
import math
import re
import  gc
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import keras
import keras.backend as KB
sys.path.append('../')

import mrcnn.model_mod as modellib
import mrcnn.visualize as visualize
import mrcnn.shapes    as shapes
from mrcnn.config      import Config
from mrcnn.model       import log
from mrcnn.dataset     import Dataset 

from mrcnn.utils       import stack_tensors, stack_tensors_3d
from mrcnn.datagen     import data_generator, load_image_gt
from mrcnn.callbacks   import get_layer_output_1,get_layer_output_2
from mrcnn.visualize   import plot_gaussian

# Root directory of the project
ROOT_DIR = os.getcwd()
MODEL_PATH = 'E:\Models'
# Directory to save logs and trained model
MODEL_DIR = os.path.join(MODEL_PATH, "mrcnn_development_logs")

# Path to COCO trained weights
COCO_MODEL_PATH   = os.path.join(MODEL_PATH, "mask_rcnn_coco.h5")
RESNET_MODEL_PATH = os.path.join(MODEL_PATH, "resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5")
print("Tensorflow Version: {}   Keras Version : {} ".format(tf.__version__,keras.__version__))


# Build configuration object -----------------------------------------------
config = shapes.ShapesConfig()
config.BATCH_SIZE      = 5                  # Batch size is 2 (# GPUs * images/GPU).
config.IMAGES_PER_GPU  = 5                  # Must match BATCH_SIZE
config.STEPS_PER_EPOCH = 2
config.FCN_INPUT_SHAPE = config.IMAGE_SHAPE[0:2]

# config.LAST_EPOCH_RAN  = 5784
config.display() 

# Build shape dataset        -----------------------------------------------
# Training dataset
# generate 500 shapes 
dataset_train = shapes.ShapesDataset()
dataset_train.load_shapes(500, config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1])
dataset_train.prepare()

# Validation dataset
dataset_val = shapes.ShapesDataset()
dataset_val.load_shapes(50, config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1])
dataset_val.prepare()

# Load and display random samples
# image_ids = np.random.choice(dataset_train.image_ids, 3)
# for image_id in [3]:
#     image = dataset_train.load_image(image_id)
#     mask, class_ids = dataset_train.load_mask(image_id)
#     visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)
print(' COCO Model Path       : ', COCO_MODEL_PATH)
print(' Checkpoint folder Path: ', MODEL_DIR)
print(' Model Parent Path     : ', MODEL_PATH)
print(' Resent Model Path     : ', RESNET_MODEL_PATH)

import pprint
pp = pprint.PrettyPrinter(indent=2, width=100)
np.set_printoptions(linewidth=100,precision=4)


# Create Model

In [ ]:
try :
    del model
    gc.collect()
except: 
    pass
KB.clear_session()
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

In [ ]:
print(' COCO Model Path       : ', COCO_MODEL_PATH)
print(' Checkpoint folder Path: ', MODEL_DIR)
print(' Model Parent Path     : ', MODEL_PATH)
print(' Resent Model Path     : ', RESNET_MODEL_PATH)
print(model.find_last())

# model.compile_only(learning_rate=config.LEARNING_RATE, layers='all')
# tst = model.keras_model.to_json()
# save_model(MODEL_DIR, 'my_saved_model')
# print(model.find_last())

In [ ]:
#model.keras_model.summary(line_length = 120) 
# model.compile_only(learning_rate=config.LEARNING_RATE, layers='heads')
# KB.set_learning_phase(1)
'''
methods to load weights
1 - load a specific file
2 - find a last checkpoint in a specific folder 
3 - use init_with keyword 
'''
## 1- look for a specific weights file 
## Load trained weights (fill in path to trained weights here)
# model_path  = 'E:\\Models\\mrcnn_logs\\shapes20180428T1819\\mask_rcnn_shapes_5784.h5'
# print(' model_path : ', model_path )
# assert model_path != "", "Provide path to trained weights"
# print("Loading weights from ", model_path)
# model.load_weights(model_path, by_name=True)    
# print('Load weights complete')

# ## 2- look for last checkpoint file in a specific folder (not working correctly)
# model.config.LAST_EPOCH_RAN = 5784
# model.model_dir = 'E:\\Models\\mrcnn_logs\\shapes20180428T1819'
# last_model_found = model.find_last()
# print(' last model in MODEL_DIR: ', last_model_found)
# # loc= model.load_weights(model.find_last()[1], by_name=True)
# # print('Load weights complete :', loc)


## 3- Use init_with keyword
## Which weights to start with?
init_with = "last"  # imagenet, coco, or last

if init_with == "imagenet":
#     loc=model.load_weights(model.get_imagenet_weights(), by_name=True)
    loc=model.load_weights(RESNET_MODEL_PATH, by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    
    # See README for instructions to download the COCO weights
    loc=model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    loc= model.load_weights(model.find_last()[1], by_name=True)
print('Load weights complete')

###  Print some model information 

In [ ]:
print('\n Inputs: ') 
for i, out in enumerate(model.keras_model.inputs):
    print(i , '    ', out)

print('\n Outputs: ') 
for i, out in enumerate(model.keras_model.outputs):
    print(i , '    ', out)

# print('\n Losses (model.metrics_names): ') 
# pp.pprint(model.get_deduped_metrics_names())
# model.keras_model.summary(line_length = 150) 

## Training

Train in two stages:
1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass `layers='heads'` to the `train()` function.

2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass `layers="all` to train all layers.

## Training head using  Keras.model.fit_generator()

In [ ]:
print(config.BATCH_SIZE)
print(model.config.BATCH_SIZE)

In [ ]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.

# model.train(dataset_train, dataset_val, 
#             learning_rate=config.LEARNING_RATE, 
# #             epochs = 69,
#             epochs_to_run =2, 
#             layers='heads')

train_layers = ['mrcnn', 'fpn','rpn']
loss_names   = [  "rpn_class_loss", "rpn_bbox_loss" , "mrcnn_class_loss", "mrcnn_bbox_loss"]
model.epoch = 3648
model.config.LEARNING_RATE = 1.0e-3
model.config.STEPS_PER_EPOCH = 7

model.train(dataset_train, dataset_val, 
            learning_rate=model.config.LEARNING_RATE, 
            epochs_to_run =352, 
#             epochs = 25,            
#             batch_size = 0
#             steps_per_epoch = 0 
            layers = train_layers,
            losses = loss_names,
            min_LR = 1.0e-6,
            )

In [ ]:
model.config.LEARNING_RATE

## - Training heads using train_in_batches ()

We need to use this method for the time being as the fit generator does not have provide EASY access to the output in Keras call backs. By training in batches, we pass a batch through the network, pick up the generated RoI detections and bounding boxes and generate our semantic / gaussian tensors ...


In [ ]:
model.train_in_batches(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE/6, 
            epochs_to_run = 3,
            layers='heads')

## Fine Tuning
Fine tune all layers

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=211,
            layers="all")

## Save 

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes.h5")
model.keras_model.save_weights(model_path)

### Define Data Generator

In [ ]:
train_generator = data_generator(dataset_train, model.config, shuffle=True,
                                 batch_size=model.config.BATCH_SIZE,
                                 augment = False)
val_generator = data_generator(dataset_val, model.config, shuffle=True, 
                                batch_size=model.config.BATCH_SIZE,
                                augment=False)

### Get next shapes from generator and display loaded shapes

In [ ]:
train_batch_x, train_batch_y = next(train_generator)

In [ ]:
# train_batch_x, train_batch_y = next(train_generator)
imgmeta_idx = model.keras_model.input_names.index('input_image_meta')
img_meta    = train_batch_x[imgmeta_idx]

for img_idx in range(config.BATCH_SIZE):
    image_id = img_meta[img_idx,0]
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    print('Image id: ',image_id)
    print('Image meta', img_meta[img_idx])
    print('Classes (1: circle, 2: square, 3: triangle ): ',class_ids)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)



### Push Data thru model using get_layer_output()

In [ ]:

model_output = get_layer_output_2(model.keras_model, train_batch_x, 1)


### Input Values

In [ ]:
# 0      Tensor("input_image:0", shape=(?, 128, 128, 3), dtype=float32)
# 1      Tensor("input_image_meta:0", shape=(?, ?), dtype=float32)
# 2      Tensor("input_rpn_match:0", shape=(?, ?, 1), dtype=int32)
# 3      Tensor("input_rpn_bbox:0", shape=(?, ?, 4), dtype=float32)
# 4      Tensor("input_gt_class_ids:0", shape=(?, ?), dtype=int32)
# 5      Tensor("input_gt_boxes:0", shape=(?, ?, 4), dtype=float32)
# 6      Tensor("input_gt_masks:0", shape=(?, 56, 56, ?), dtype=bool)

input_image_meta =  train_batch_x[1]
input_rpn_match  =  train_batch_x[2]
input_rpn_bbox   =  train_batch_x[3]
input_gt_class_ids = train_batch_x[4]
input_gt_bboxes = train_batch_x[5]
# gt_masks   =  train_batch_x[6]
print(' input_rpn_match    ', input_rpn_match.shape)
print(' input_rpn_bbox     ', input_rpn_bbox.shape)
print(' input_gt_class_ids ', input_gt_class_ids.shape)
print(' input_gt_bboxes    ', input_gt_bboxes.shape)


### Output Values

In [ ]:
# KB.set_session(sess)
print(len(model_output))
# rpn_class_logits   = model_output[0]
# rpn_class          = model_output[1]
# rpn_bbox           = model_output[2]
# rpn_proposal_rois  = model_output[3]
output_rois        = model_output[4]
target_class_ids   = model_output[5]
target_bbox_deltas = model_output[6]
roi_gt_boxes       = model_output[7]
mrcnn_class_logits = model_output[8]
mrcnn_class        = model_output[9]
mrcnn_bbox         = model_output[10]
# rpn_class_loss   = model_output[11]
# rpn_bbox_loss    = model_output[12]
# mrcnn_class_loss = model_output[13]
# mrcnn_bbox_loss  = model_output[14]
fcn_bbox_loss      = model_output[15]
pred_hm            = model_output[16]
gt_hm              = model_output[17]
pred_hm_norm       = model_output[18]
gt_hm_norm         = model_output[19]
pred_tensor        = model_output[20]
gt_tensor          = model_output[21]
gt_deltas          = model_output[22]
fcn_heatmap        = model_output[23]
fcn_class_logits   = model_output[24]
fcn_scores         = model_output[25]
fcn_bbox_deltas    = model_output[26]
# print(type(model_output[4]))
# print(type(output_rois))

In [ ]:
print(gt_deltas.shape, fcn_bbox_deltas.shape)

In [ ]:
img = 0
cls = 3
print(gt_deltas[img,cls])
print(fcn_bbox_deltas[img,cls])

In [ ]:
model.keras_model.summary(line_length=132, positions=[0.30,0.75, .83, 1. ])

In [ ]:
print(model.keras_model.summary)